<a href="https://colab.research.google.com/github/MubshAli/StockEval/blob/main/intrinsicValue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yahoo_fin --upgrade

     |████████████████████████████████| 81 kB 5.8 MB/s 
     |████████████████████████████████| 83 kB 2.4 MB/s 
     |████████████████████████████████| 103 kB 71.8 MB/s 
     |████████████████████████████████| 127 kB 71.2 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=c4de6c6c5340c9a6eaa90349d22b8b054bc769d47c904473df590fa02da9b50f
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=089c5bfb8901486bdf98c011d40e3c53bdf1ce6e96ab76bb167ab2fd1cb61ad1
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=e2501a40124e0d4de3d9fc6ecdc45e5c22860daf4019eb2964e2a85dac7133a2
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36ce

In [ ]:
pip install requests_html

In [ ]:
from yahoo_fin.stock_info import get_data
#get_analysts_info()

In [ ]:
import yahoo_fin.stock_info as si

{'1y Target Est': 167.13,
 '52 Week Range': '103.10 - 157.26',
 'Ask': '148.72 x 1100',
 'Avg. Volume': 78099357.0,
 'Beta (5Y Monthly)': 1.2,
 'Bid': '148.71 x 2200',
 "Day's Range": '148.70 - 155.48',
 'EPS (TTM)': 5.11,
 'Earnings Date': 'Oct 27, 2021 - Nov 01, 2021',
 'Ex-Dividend Date': 'Aug 06, 2021',
 'Forward Dividend & Yield': '0.88 (0.59%)',
 'Market Cap': '2.463T',
 'Open': 155.0,
 'PE Ratio (TTM)': 29.16,
 'Previous Close': 154.07,
 'Quote Price': 148.97000122070312,
 'Volume': 140893235.0}

,Growth Estimates,AAPL,Industry,Sector(s),S&P 500
0,Current Qtr.,68.50%,NaN,NaN,NaN
1,Next Qtr.,11.30%,NaN,NaN,NaN
2,Current Year,70.40%,NaN,NaN,NaN
3,Next Year,2.10%,NaN,NaN,NaN
4,Next 5 Years (per annum),19.61%,NaN,NaN,NaN
5,Past 5 Years (per annum),8.42%,NaN,NaN,NaN


In [ ]:
from pandas import *

In [ ]:
x

,Growth Estimates,AAPL,Industry,Sector(s),S&P 500
0,Current Qtr.,68.50%,NaN,NaN,NaN
1,Next Qtr.,11.30%,NaN,NaN,NaN
2,Current Year,70.40%,NaN,NaN,NaN
3,Next Year,2.10%,NaN,NaN,NaN
4,Next 5 Years (per annum),19.61%,NaN,NaN,NaN
5,Past 5 Years (per annum),8.42%,NaN,NaN,NaN


1961
10000
0.1961


In [ ]:
pip install get-all-tickers

  Created wheel for get-all-tickers: filename=get_all_tickers-1.8-py3-none-any.whl size=4247 sha256=eb9c151df3c9c4a3b70c565f1977f0a0f6c402a31b334de33744563edbad591d
  Stored in directory: /root/.cache/pip/wheels/f0/8d/8f/25b9632788de8771fb8be22529bbe0ba5ae5446f3be65b1ab3
Successfully built get-all-tickers


In [ ]:
from get_all_tickers import get_tickers as gt

In [ ]:
s = si.tickers_dow()
m = si.tickers_nasdaq()
a = si.tickers_sp500()
o = si.tickers_other()
b = s + a + m + o
len(b)


,Growth Estimates,AAPL,Industry,Sector(s),S&P 500
0,Current Qtr.,68.50%,NaN,NaN,NaN
1,Next Qtr.,11.30%,NaN,NaN,NaN
2,Current Year,70.40%,NaN,NaN,NaN
3,Next Year,2.10%,NaN,NaN,NaN
4,Next 5 Years (per annum),19.61%,NaN,NaN,NaN
5,Past 5 Years (per annum),8.42%,NaN,NaN,NaN


In [ ]:
result = []
for i in range(10):
  company = b[i]
  #print(company)
  quote_table = si.get_quote_table(company, dict_result=True)
  anal= si.get_analysts_info(company)
  x = anal['Growth Estimates']
  val = x[company].values[4]
  print(val)
  s = ""  
  for i in range(len(val)):
    if val[i] == "%" or val[i] == ".":
      s = s
    else:
      s += val[i]
  #print(s)
  l = len(s)
  div = 10000
  #print(div)
  s = int(s)
  s = s/div
  print(s)

  growth_rate = s
  min_ror = 0.15
  margin_of_safety = 0.50
  p_e_ratio = quote_table["PE Ratio (TTM)"]
  eps = quote_table["EPS (TTM)"]
  curr_price = quote_table["Quote Price"]
  eps_after_ten = eps
  for i in range(9):
    eps_after_ten += growth_rate*eps_after_ten
  share_price_after_ten = eps_after_ten * p_e_ratio
  expected_price = share_price_after_ten
  for i in range(9):
    expected_price = expected_price/(1+min_ror)
  curr_value = expected_price
  #print(curr_value)
  add_to_result = (company, curr_value)
  print(add_to_result)
  result.append(add_to_result)




19.61%
0.1961
('AAPL', 212.2131591215428)
5.95%
0.0595
('AMGN', 102.16652518029487)
41.00%
0.41
('AXP', 993.8677031716978)
12.33%
0.1233
('BA', nan)
31.13%
0.3113
('CAT', 667.6163349679173)
10.37%
0.1037
('CRM', 177.68017659665324)
6.45%
0.0645
('CSCO', 28.89272429356664)
-4.90%
-0.049
('CVX', nan)
50.89%
0.5089
('DIS', nan)
-5.43%
-0.0543
('DOW', 10.400673237317468)


In [ ]:
q = si.get_quote_table("BA",dict_result =True)


In [ ]:
import math
print(q)


growth_rate = 0.15
min_ror = 0.15
margin_of_safety = 0.50
p_e_ratio = q["PE Ratio (TTM)"]

eps = q["EPS (TTM)"]
x = float('nan')
if(math.isnan(p_e_ratio)):
  print("GG")
curr_price = q["Quote Price"]
eps_after_ten = eps
for i in range(9):
  eps_after_ten += growth_rate*eps_after_ten
share_price_after_ten = eps_after_ten * p_e_ratio
expected_price = share_price_after_ten
for i in range(9):
  expected_price = expected_price/(1+min_ror)
curr_value = expected_price



#print(curr_value)
# add_to_result = (company, curr_value)
# print(add_to_result)
# result.append(add_to_result)

{'1y Target Est': 268.22, '52 Week Range': '141.58 - 278.57', 'Ask': '211.38 x 900', 'Avg. Volume': 10777435.0, 'Beta (5Y Monthly)': 1.62, 'Bid': '211.30 x 1200', "Day's Range": '210.13 - 215.89', 'EPS (TTM)': -20.55, 'Earnings Date': 'Oct 26, 2021 - Nov 01, 2021', 'Ex-Dividend Date': 'Feb 13, 2020', 'Forward Dividend & Yield': 'N/A (N/A)', 'Market Cap': '122.986B', 'Open': 215.16, 'PE Ratio (TTM)': nan, 'Previous Close': 213.94, 'Quote Price': 210.3000030517578, 'Volume': 6959706.0}
GG


False

In [ ]:
print(quote_table)

{'1y Target Est': 67.29, '52 Week Range': '44.33 - 71.38', 'Ask': '60.68 x 800', 'Avg. Volume': 4141314.0, 'Beta (5Y Monthly)': 1.72, 'Bid': '60.67 x 1400', "Day's Range": '60.34 - 61.44', 'EPS (TTM)': 2.65, 'Earnings Date': 'Oct 20, 2021 - Oct 25, 2021', 'Ex-Dividend Date': 'Aug 30, 2021', 'Forward Dividend & Yield': '2.80 (4.38%)', 'Market Cap': '45.126B', 'Open': 60.85, 'PE Ratio (TTM)': 22.82, 'Previous Close': 60.28, 'Quote Price': 60.40999984741211, 'Volume': 4093257.0}
